In [ ]:
! pip install transformers datasets
!pip install pyserini
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.9 MB/s eta 0:00:00
  

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="saemhasan/our_bertserini")

In [ ]:
# qa_pipeline = pipeline(
#   "question-answering",
#   model="csarron/bert-base-uncased-squad-v1",
#   tokenizer="csarron/bert-base-uncased-squad-v1"
# )

# question_answerer = qa_pipeline

## DATASET

In [ ]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:20000]")

squad_validation = load_dataset('squad', split='validation[:]')

# print(squad_validation[0])

Retriver

In [ ]:
from pyserini.search.lucene import LuceneSearcher
def get_searcher():
    try:
        return LuceneSearcher('indexes/lucene-index.enwiki-20180701-paragraphs')
    except Exception as e:
        # print(e)
        print('Index not found, downloading prebuilt index...')
        return LuceneSearcher.from_prebuilt_index('enwiki-paragraphs')

searcher = get_searcher()


Index not found, downloading prebuilt index...




lucene-index.enwiki-20180701-paragraphs.tar.gz: 0.00B [00:00, ?B/s]

lucene-index.enwiki-20180701-paragraphs.tar.gz:   0%|          | 8.00k/16.5G [00:00<171:20:56, 28.7kB/s]

lucene-index.enwiki-20180701-paragraphs.tar.gz:   0%|          | 152k/16.5G [00:00<9:55:31, 496kB/s]    

lucene-index.enwiki-20180701-paragraphs.tar.gz:   0%|          | 1.29M/16.5G [00:00<1:14:06, 3.99MB/s]

lucene-index.enwiki-20180701-paragraphs.tar.gz:   0%|          | 7.02M/16.5G [00:00<14:21, 20.6MB/s]  

lucene-index.enwiki-20180701-paragraphs.tar.gz:   0%|          | 13.9M/16.5G [00:00<08:29, 34.8MB/s]

lucene-index.enwiki-20180701-paragraphs.tar.gz:   0%|          | 21.8M/16.5G [00:00<06:11, 47.6MB/s]

lucene-index.enwiki-20180701-paragraphs.tar.gz:   0%|          | 29.9M/16.5G [00:00<05:11, 56.8MB/s]

lucene-index.enwiki-20180701-paragraphs.tar.gz:   0%|          | 38.2M/16.5G [00:01<04:38, 63.5MB/s]

lucene-index.enwiki-20180701-paragraphs.tar.gz:   0%|          | 46.8M/16.5G [00:01<04:08, 71.0MB/s]


In [ ]:
def predict_bert(question, context):
  return  question_answerer(question=question, context=context)


In [ ]:
import json
kotogula_paragraph = 10
mu = .95

def predict(question, verbose=False):
  hits = searcher.search(question, k = kotogula_paragraph)
  results = []

  for i, hit in enumerate(hits):
    hit_context, hit_score  = hit.raw,  hit.score 
    result = question_answerer(question=question, context=hit_context)

    # print(f"Answer: '{result['answer']}', bert_score: {round(result['score'], 4)}, arserini_score: {hit.score} start: {result['start']}, end: {result['end']}")
    # print('\n\n', hit.raw)

    result['total_score'] = mu * result['score'] + ( 1 - mu ) * hit_score 
    # result['lucene_document'] = hit.lucene_document
    result['raw'] = hit_context


    results.append(result)
    

  results = sorted(results, key = lambda x : x['total_score'])

  if verbose:
    for result in results:
      print(result['answer'], result['total_score'], result['score'],  result['raw'],  '\n')

  # print('=================== final answer ======================\n\n')
  return results[-1]['answer']

question = 'What is the capital of  Oklahoma?'

predict(question, True)

Oklahoma 0.4287036066195484 1.3065036164050525e-08 Henry Marshall Furman . Fourth. If it be conceded that the Governor's position is correct, and that he has the right to suspend the execution of any provision of law of which he may not approve; and if it be true that the other officials of the state are answerable to him, and not to the people — then we have an empire in Oklahoma, and not a free state. This would establish a precedent which would justify any subsequent Governor, who might be opposed to the prohibitory liquor law, in commuting all jail or penitentiary sentences inflicted in such cases upon the ground that he did not like the law, and that he knew better than the people what should be done in such cases. The same principle would apply to all laws. Concede the principle contended for by the Governor, and where will the matter end? It would utterly demoralize the enforcement of law in Oklahoma, and would convert the state government into one of men and not of law. What do

'Guthrie'

# Performance

In [ ]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    f1 =  2 * (prec * rec) / (prec + rec)

    return f1

In [ ]:
def get_gold_answers(example):
    """helper function that retrieves all possible true answers from a squad2.0 example"""
    
    gold_answers = [answer for answer in example['answers']['text'] if answer]

    # if gold_answers doesn't exist it's because this is a negative example - 
    # the only correct answer is an empty string
    if not gold_answers:
        gold_answers = [""]
        
    return gold_answers

In [ ]:
question = 'What date is the victory day of Bangladesh?'

answer = predict(question, verbose=True)

September 0.545248593330553 4.807522600458469e-06 Bangladesh national cricket team . Bangladesh played 7 more Tests and 11 more ODIs in 2003 against Australia, Pakistan and England, losing every single match. However, in September, Bangladesh came very close to its first Test victory, when it lost to Pakistan by only one wicket. It was just the tenth time in Test history that a team had lost by a single wicket. In series against Pakistan, Alok Kapali became the first Bangladesh player to take a hat-trick in Test cricket, dismissing Shabbir Ahmed, Danish Kaneria, and Umar Gul. When England toured Bangladesh October and November, Bangladesh's captain, Mahmud, was booed. Ahead of Bangladesh's tour of Zimbabwe in February and March the next year, he was dropped from the squad and batsman Habibul Bashar was granted the captaincy. At the time, Zimbabwe were without many of their senior players. Bangladesh lost the Test series 1–0, and the ODI series 2–1. The second Test was drawn after 3 day

In [ ]:


def compute_indvidual(squad_record, verbose=False):
  question = squad_record['question']
  gold_answers = get_gold_answers(squad_record)

  prediction = predict(question, verbose)

  em_score = max((compute_exact_match(prediction, answer)) for answer in gold_answers)
  f1_score = max((compute_f1(prediction, answer)) for answer in gold_answers)

  if verbose:
    print(f"Question: {question}")
    print(f"Prediction: {prediction}")
    print(f"Given Context: {squad_record['context']}")
    print(f"True Answers: {gold_answers}")
    print(f"EM: {em_score} \t F1: {f1_score}")
  
  return em_score, f1_score



# for squad_record in squad_validation[454]:
  # compute_indvidual(squad_record, True)
  # break



In [ ]:

def compute_indvidual_bert(squad_record, verbose=False):
  question = squad_record['question']
  context = squad_record['context']
  gold_answers = get_gold_answers(squad_record)

  prediction = predict_bert(question, context)['answer']

  em_score = max((compute_exact_match(prediction, answer)) for answer in gold_answers)
  f1_score = max((compute_f1(prediction, answer)) for answer in gold_answers)

  if verbose:
    print(f"Question: {question}")
    print(f"Prediction: {prediction}")
    print(f"Given Context: {squad_record['context']}")
    print(f"True Answers: {gold_answers}")
    print(f"EM: {em_score} \t F1: {f1_score}")
  
  return em_score, f1_score

In [ ]:
compute_indvidual(squad_validation[1879], True)


Academic programs 0.5412438899074918 0.0008054563659243286 Dietitian . Academic programs, professional associations, and credentialing requirements do not exist for dietitians in most other African countries, where the number of professionals in dietetics is very low. 

the government, a higher education institution or a private source 0.9139310836791996 0.3708019256591797 Certified teacher . A certified teacher is a teacher who has earned credentials from an authoritative source, such as the government, a higher education institution or a private source. This teacher qualification gives a teacher authorization to teach and grade in pre-schools, primary or secondary education in countries, schools, content areas or curricula where authorization is required. While many authorizing entities require student teaching before earning teacher certification, routes vary from country to country. A teaching qualification is one of a number of academic and professional degrees that enables a pers

(0, 0)

In [ ]:
compute_indvidual(squad_validation[502], True)


Los Angeles Memorial Coliseum. 0.6225413572158236 2.628521045267007e-08 Super Bowl LIV . On May 19, 2015, the league announced the four finalists that will compete to host either Super Bowl LIII in 2019 or Super Bowl LIV. NFL owners voted on these cities in May 2016, with the first round of voting determining who will host Super Bowl LIII, and the second round deciding the site for Super Bowl LIV. The league had also originally announced in 2015 that Los Angeles would be eligible as a potential Super Bowl LIV site if there is a stadium in place, and a team moved there by the start of the 2018 season. The league opened the relocation window in January 2016, selecting the former St. Louis Rams to return to Los Angeles; their new stadium in Inglewood, California was, at the time of the vote, not scheduled to open until August 2019 (it began construction in December 2016, giving nearly three years to construct the stadium). This meant that the new stadium was scheduled to be open in time f

(1, 1.0)

In [ ]:
compute_indvidual(squad_validation[6935], True)


boutique condominium 0.6814723917224916 0.0012977445730939507 15 Union Square West . Brack Capital Real Estate purchased the property in 2006, and restored the original six-story structure and added six newly constructed floors to create a boutique condominium with 36 residences. The brick façade was dismantled and the original arches were reconditioned and wrapped behind a façade of glass and black anodized aluminum. The original structure was topped by an additional six stories of all glass residences. Designed by Eran Chen of ODA-Architecture, previously of Perkins Eastman, the building blends historic and contemporary elements. 

From 1939-1940 0.7205406444147235 0.009212922304868698 State Farm Downtown Building . The first eight-floor section of the building (the south portion) was constructed in 1929. In 1934, an additional four floors and a penthouse were added. Five years later, State Farm acquired the property directly to the north of the lot the Downtown Building was construc

(1, 1.0)

In [ ]:
compute_indvidual(squad_validation[10088], True)


Stroud was never convicted of murder 0.63254938154923 3.0855359933745774e-10 Characters of Carnivàle . Varlyn Stroud is a convict. His original biography gives insight into his criminal past and names Stroud's sister Clara as his first murder. When Clara was nine years old, Stroud deliberately upset a Ferris wheel car so Clara fell to her death. Stroud's second murder was his grandfather three years later, which he covered as a hunting accident. Having possibly committed over a dozen contract-killings, Stroud was never convicted of murder, yet still spent most of his life in prison on a variety of charges. Stroud's story arc in the series begins in season 2 when he hears Brother Justin's radio sermon. Made Brother Justin's apostle in a subliminal message, Stroud escapes prison to find and bring Henry Scudder to Brother Justin. Stroud first tracks Scudder's old life and later follows the route of the carnival. With Ben's unintentional aid, Stroud finally finds the man he is looking for.

(1, 1.0)

# Our Query

In [ ]:
question = 'who won FIFA men\'s world cup 2014'

predict(question, verbose = True)

Argentina 0.8267722957767554 0.018833046779036522 2014 FIFA World Cup knockout stage . The two teams had met in eight previous matches, including four times in the FIFA World Cup: Netherlands won 4–0 in the second group stage of the 1974 FIFA World Cup and 2–1 in the quarter-finals of the 1998 FIFA World Cup, Argentina won 3–1 (after extra time) in the final of the 1978 FIFA World Cup, where as a group stage encounter of the 2006 FIFA World Cup ended in a 0–0 draw. 

FIBA Women's Basketball World Cup 0.8352095046546317 0.007312715984880924 FIBA Women's Basketball World Cup . Formerly known as the FIBA World Championship for Women, the name changed shortly after its 2014 edition. From 1986 through 2014, the tournament was held in the same year as the men's FIBA Basketball World Cup, though in different countries. After the 2014 editions of both championships, the men's event was rescheduled on a new four-year cycle (the next in September 2019) to avoid conflict with the men's FIFA World

'Germany'

In [ ]:
question = 'when was wikipedia launched?'

predict(question, verbose = False)

'February 2010'

In [ ]:
question = 'when was wikipedia first launched?'

predict(question, verbose = False)

'January 2001'

In [ ]:
predict('What is the mother tongue of bangladeshi people?', verbose=False)

'Bangla'

In [ ]:
predict('What is the victory Day of Bangladesh?', verbose=False)

'Bijôy Dibôs'

In [ ]:
predict('What date is the victory Day of Bangladesh?', verbose=False)

# Statistics

In [ ]:
from numpy import random

indexes = random.randint(len(squad_validation), size=1)
indexes

In [ ]:
# compute_indvidual(squad_validation[5557], True)


In [ ]:
em_score , f1_score = 0, 0 

_cnt = 0

from tqdm.notebook import tqdm
for index in tqdm(indexes):

  # print(squad_record)
  try:

    _em_score , _f1_score=compute_indvidual_bert(squad_validation[int(index)])
    
    em_score += _em_score
    f1_score += _f1_score

    _cnt += 1 
  except Exception as e:
    print(e)
    break
  
  # if _cnt > 100: 
  #   break


print(f"EM : {em_score / _cnt * 100}, f1 : {f1_score/_cnt * 100}, _total : {_cnt}")


In [ ]:
# em_score , f1_score = 0, 0 

# _cnt = 0

# import tqdm
# for index in tqdm(indexes):

#   # print(squad_record)
#   try:

#     predict(squad_validation[index])
#     _em_score , _f1_score=compute_indvidual(squad_record)
    
#     em_score += _em_score
#     f1_score += _f1_score

#     _cnt += 1 
#   except Exception as e:
#     print(e)
#     pass
  
#   if _cnt > 10: 
#     break

In [ ]:
em_score , f1_score = 0, 0 

_cnt = 0

from tqdm.notebook import tqdm
for index in tqdm(indexes):

  # print(squad_record)
  try:
    squad_re...................................vvvvvvvvvvvvvvv   cord = squad_validation[int(index)]
    # predict(squad_validation[int(index)])
    _em_score , _f1_score=compute_indvidual(squad_record)
    
    em_score += _em_score
    f1_score += _f1_score

    _cnt += 1 
  except Exception as e:
    print(e)
    pass
  
  # if _cnt > 10: 
  #   break

  


print(f"EM : {em_score / _cnt * 100}, f1 : {f1_score/_cnt * 100}, _total : {_cnt}")


In [ ]:
# print(em_score / _cnt * 100, f1_score/_cnt * 100, _cnt)